In [ ]:
## Load R Libraries : R-4.0 (at Broad)
library(data.table)
library(survival)
# install.packages("survminer")
#library(survminer)
library(coxphw)
library(stringr)
library(dplyr)
library(tidyr)
# library(readxl)
library(ggplot2)
library(ggpubr)
library(cowplot)
theme_set(theme_cowplot())

# set working directory
setwd("/medpop/esp2/mesbah/projects/ch_progression/aric/epi/")

In [ ]:
#
    # qcd CH variants with phenotypes
ch_var_in_v05_qcd.pheno <- fread("../pheno/ch_var_in_v05_qcd_pheno.23Mar2023.csv", header = T)

# Load data
aric_baseline_n_v05 <- fread("../pheno/aric_baseline_n_v05_N4189.pheno_ch_status_trajectory.23Mar2023.csv", header=T)

aric_baseline_n_v05$dAge <- aric_baseline_n_v05$Age - aric_baseline_n_v05$age_base
summary(aric_baseline_n_v05$dAge)

com.expansion.CH_v_b_v5_all <- fread("../pheno/combined.expansion.CH_v_b_v5_all.growth_rate.23Mar2023.csv", header=T)
  # smoking: never=3, former=2, current=1 
aric_baseline_n_v05$cig_base_fact <- factor(aric_baseline_n_v05$cig_base, 
                                            levels = c(3,2,1))
######## composite CHD
# chd_is_base== CHD or IS
aric_baseline_n_v05$chd_is_base <- ifelse(aric_baseline_n_v05$chd_base==1 | aric_baseline_n_v05$is_base==1, 1,
                                          ifelse(aric_baseline_n_v05$chd_base==0 | aric_baseline_n_v05$is_base==0,0,NA))
table(aric_baseline_n_v05$chd_is_base, exclude = NULL)
# ASCVD = c("chd", "is", "hf")
aric_baseline_n_v05$ascvd_base <- ifelse(aric_baseline_n_v05$chd_base==1 | 
                                           aric_baseline_n_v05$is_base==1 | 
                                           aric_baseline_n_v05$hf_base==1, 1,
                                         ifelse(aric_baseline_n_v05$chd_base==0 | 
                                                  aric_baseline_n_v05$is_base==0 | 
                                                  aric_baseline_n_v05$hf_base==0,0,NA))
table(aric_baseline_n_v05$ascvd_base, exclude = NULL)
aric_baseline_n_v05$ascvd_base[is.na(aric_baseline_n_v05$ascvd_base)] <- 0
table(aric_baseline_n_v05$ascvd_base, exclude = NULL)


##### Age at Detection: >=75 or less
aric_baseline_n_v05$Age_at_incidence <- ifelse(aric_baseline_n_v05$Age>=75,">=75", "<75")
table(aric_baseline_n_v05$Age_at_incidence)



In [ ]:

##### Survival plots

## P = 0.002 log-rank
png("~/Documents/Project/Baylor_ARIC_Exomes/Paper1_ARIC/epi/inident_SF_stratified_by_age.png",
    width=8, height=6, units= "in", res=300, pointsize = 4)

b <- ggsurvplot(survfit(Surv(dAge, incident_SF) ~ Age_at_incidence, 
                   data = aric_baseline_n_v05), 
           data = aric_baseline_n_v05,
           fun = 'event' , 
           xlab="Years", ylab="Cumulative incidence of SF",
           legend.title="Age at detection", 
           legend.labs = c("<75", ">=75"), title="b",
           pval = F, pval.method = F)
dev.off()
## P= 0.011
png("~/Documents/Project/Baylor_ARIC_Exomes/Paper1_ARIC/epi/inident_CH_stratified_by_age.png",
    width=8, height=6, units= "in", res=300, pointsize = 4)
a <- ggsurvplot(survfit(Surv(dAge, incident_CH) ~ Age_at_incidence, 
                   data = aric_baseline_n_v05), 
           data = aric_baseline_n_v05,
           fun = 'event' , title="a",
           xlab="Years", ylab="Cumulative incidence of CH",
           legend.title="Age at detection", 
           legend.labs = c("<75", ">=75"), 
           pval = F, pval.method = F)
dev.off()

c <- 
  
  
png("~/Documents/Project/Baylor_ARIC_Exomes/Paper1_ARIC/epi/incident_CH_SF.by_detection_Age.png",
    width=16, height=8, units= "in", res=300, pointsize = 4)

ggarrange(a, b, c, d, ncol = 2, nrow = 2 )

dev.off()
